In [1]:
import pandas as pd
from sklearn.model_selection import  train_test_split

In [3]:
#Read Data
X_full = pd.read_csv('train.csv', index_col='Id') #train data set
X_test_full = pd.read_csv('test.csv', index_col='Id') #test data set

In [4]:
#obtain Target i.e what we want to predict
y = X_full['SalePrice']

#obtain features that would be used to predict the target
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = X_full[features].copy()
X_test = X_test_full[features].copy()
